**This notebook is an exercise in the [Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/manipulating-geospatial-data).**

---


# Introduction

You are a Starbucks big data analyst ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

<center>
<img src="https://i.imgur.com/BIyE6kR.png" width="450"><br/><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [2]:
import math
import pandas as pd
import geopandas as gpd
#from geopy.geocoders import Nominatim            # What you'd normally run
from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex4 import *

이전 연습의 'embed_map()' 함수를 사용하여 지도를 시각화합니다.

In [3]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Geocode the missing locations.

다음 코드 셀을 실행하여 캘리포니아 주에 있는 Starbucks 위치를 포함하는 DataFrame 'starbucks'를 만듭니다.

In [4]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("../input/geospatial-learn-course-data/starbucks_locations.csv")
starbucks.head()

대부분의 상점은 (위도, 경도) 위치를 알고 있습니다. 그러나 버클리시의 모든 위치가 누락되었습니다.

In [5]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

아래 코드 셀을 사용하여 Nominatim 지오코더로 이 값을 채우십시오.

튜토리얼에서 우리는 값을 지오코딩하기 위해 `Nominatim()`(`geopy.geocoders`에서)을 사용했으며 이것은 이 과정 이외의 자신의 프로젝트에서 사용할 수 있는 것입니다.

이 연습에서는 약간 다른 함수 `Nominatim()`(`learntools.geospatial.tools`에서)을 사용합니다. 이 기능은 노트북 상단에서 가져온 것으로 GeoPandas의 기능과 동일하게 작동합니다.

즉,
- 노트북 상단의 import 문을 변경하지 않고,
- 아래 코드 셀에서 지오코딩 함수를 `geocode()`로 호출합니다.

In [6]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# Your code here
def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)
# Check your answer
q_1.check()

In [ ]:
# Line below will give you solution code
#q_1.solution()

### 2) View Berkeley locations.

방금 찾은 위치를 살펴보겠습니다. OpenStreetMap 스타일로 버클리의 (위도, 경도) 위치를 시각화합니다

In [7]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Uncomment to see a hint
#q_2.a.hint()

# Show the map
embed_map(m_2, 'q_2.html')

In [ ]:
# Get credit for your work after you have created a map
#q_2.a.check()

# Uncomment to see our solution (your code may look different!)
#q_2.a.solution()

버클리의 5개 위치만 고려하면, (위도, 경도) 위치 중 몇 개(위도, 경도)가 잠재적으로 올바른 것처럼 보입니까(올바른 도시에 위치)?
##### 5개 위치가 모두 맞는 것 같습니다

In [ ]:
# View the solution (Run this code cell to receive credit!)
#q_2.b.solution()

### 3) Consolidate your data.

아래 코드를 실행하여 캘리포니아 주의 각 카운티에 대한 이름, 면적(제곱 킬로미터 단위) 및 고유 ID("GEOID" 열에 있음)가 포함된 GeoDataFrame `CA_counties`를 로드합니다. "형상" 열에는 카운티 경계가 있는 다각형이 포함되어 있습니다.

In [8]:
CA_counties = gpd.read_file("../input/geospatial-learn-course-data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

다음으로 3개의 DataFrame을 생성합니다.
- 'CA_pop'은 각 카운티의 인구 추정치를 포함합니다.
- `CA_high_earners`에는 연간 소득이 $150,000 이상인 가구 수가 포함됩니다.
- 'CA_median_age'는 각 카운티의 중위 연령을 포함합니다.

In [9]:
CA_pop = pd.read_csv("../input/geospatial-learn-course-data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("../input/geospatial-learn-course-data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("../input/geospatial-learn-course-data/CA_county_median_age.csv", index_col="GEOID")

다음 코드 셀을 사용하여 `CA_counties` GeoDataFrame을 `CA_pop`, `CA_high_earners` 및 `CA_median_age`와 결합합니다.

결과 GeoDataFrame의 이름을 `CA_stats`로 지정하고 "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners" 및 "median_age"의 8개 열이 있는지 확인합니다. 또한 CRS가 `{'init': 'epsg:4326'}`으로 설정되어 있는지 확인하십시오.

In [ ]:

from pyproj import CRS
CRS("EPSG:4326")
CA_stats.crs = {'init': 'epsg:4326'}

In [12]:
# Your code here
from pyproj import CRS
CA_stats.crs = {'init': 'epsg:4326'}
CA_stats = cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

# Check your answer
q_3.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

이제 모든 데이터가 한 곳에 있으므로 열 조합을 사용하는 통계를 훨씬 쉽게 계산할 수 있습니다. 다음 코드 셀을 실행하여 인구 밀도가 있는 "밀도" 열을 만듭니다.

In [13]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### 4) Which counties look promising?

모든 정보를 단일 GeoDataFrame으로 축소하면 특정 기준을 충족하는 카운티를 훨씬 더 쉽게 선택할 수 있습니다.

다음 코드 셀을 사용하여 `CA_stats` GeoDataFrame에서 행(및 모든 열)의 하위 집합을 포함하는 GeoDataFrame `sel_counties`를 만듭니다. 특히 다음과 같은 카운티를 선택해야 합니다.
- 연간 소득이 15만 달러 이상인 가구가 10만 가구 이상인 경우
- 중위 연령이 38.5세 미만이고,
- 주민 밀도는 최소 285명(제곱 킬로미터당)입니다.

또한 선택한 카운티는 다음 기준 중 하나 이상을 충족해야 합니다.
- 연간 $150,000를 버는 최소 500,000 가구가 있고,
- 중위 연령이 35.5세 미만이거나
- 주민 밀도는 최소 1400명(제곱 킬로미터당)입니다.

In [15]:
# Your code here
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]

# Check your answer
q_4.check()

In [14]:
# Lines below will give you a hint or solution code
#q_4.hint()
#q_4.solution()

### 5) How many stores did you identify?

다음 Starbucks Reserve Roastery 위치를 찾을 때 선택한 카운티 내의 모든 매장을 고려하고 싶습니다. 그렇다면 선택한 카운티 내에 몇 개의 매장이 있습니까?

이 질문에 답할 준비를 하려면 다음 코드 셀을 실행하여 모든 스타벅스 위치가 포함된 GeoDataFrame `starbucks_gdf`를 만듭니다.

In [19]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

그렇다면 선택한 카운티에는 몇 개의 매장이 있습니까?

In [20]:
# Fill in your answer

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)


# Check your answer
q_5.check()

In [21]:
# Lines below will give you a hint or solution code
#q_5.hint()
#q_5.solution()

### 6) Visualize the store locations.

이전 질문에서 식별한 상점의 위치를 ​​보여주는 지도를 만드십시오.

In [23]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

# Uncomment to see a hint
#q_6.hint()

# Show the map
embed_map(m_6, 'q_6.html')

In [24]:
# Get credit for your work after you have created a map
q_6.check()

# Uncomment to see our solution (your code may look different!)
#q_6.solution()

# Keep going

Learn about how **[proximity analysis](https://www.kaggle.com/alexisbcook/proximity-analysis)** can help you to understand the relationships between points on a map.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*